## Key Objective:

#### To demonstrate how one can use python (jupyter notebooks) to combine multiple files (e.g.excel in this case) for easier data manipulation, cleaning and analysis

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
!ls ../Combining_Multiple_Files

data         script.ipynb


In [3]:
!ls ../Combining_Multiple_Files/data

customer-status.xlsx sales-jan-2014.xlsx
sales-feb-2014.xlsx  sales-mar-2014.xlsx


In [4]:
import glob

In [5]:
glob.glob("../Combining_Multiple_Files/data/sales*.xlsx")

['../Combining_Multiple_Files/data/sales-feb-2014.xlsx',
 '../Combining_Multiple_Files/data/sales-mar-2014.xlsx',
 '../Combining_Multiple_Files/data/sales-jan-2014.xlsx']

#### Initialize an empty dataframe and to be used in appending all the individual files

In [6]:
all_data = pd.DataFrame()
for file in glob.glob("../Combining_Multiple_Files/data/sales*.xlsx"):
    df = pd.read_excel(file)
    all_data = all_data.append(df,ignore_index=True)

In [7]:
all_data.describe()

,account number,quantity,unit price,ext price
count,384.000000,384.000000,384.000000,384.000000
mean,478125.989583,24.372396,56.651406,1394.517344
std,220902.947401,14.373219,27.075883,1117.809743
min,141962.000000,-1.000000,10.210000,-97.160000
25%,257198.000000,12.000000,32.612500,482.745000
50%,424914.000000,23.500000,58.160000,1098.710000
75%,714466.000000,37.000000,80.965000,2132.260000
max,786968.000000,49.000000,99.730000,4590.810000


In [8]:
all_data.head()

,account number,name,sku,quantity,unit price,ext price,date
0,383080,Will LLC,B1-20000,7,33.69,235.83,2014-02-01 09:04:59
1,412290,Jerde-Hilpert,S1-27722,11,21.12,232.32,2014-02-01 11:51:46
2,412290,Jerde-Hilpert,B1-86481,3,35.99,107.97,2014-02-01 17:24:32
3,412290,Jerde-Hilpert,B1-20000,23,78.90,1814.70,2014-02-01 19:56:48
4,672390,Kuhn-Gusikowski,S1-06532,48,55.82,2679.36,2014-02-02 03:45:20


In [9]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384 entries, 0 to 383
Data columns (total 7 columns):
account number    384 non-null int64
name              384 non-null object
sku               384 non-null object
quantity          384 non-null int64
unit price        384 non-null float64
ext price         384 non-null float64
date              384 non-null object
dtypes: float64(2), int64(2), object(3)
memory usage: 21.1+ KB


#### Convert date column from object to datetime 

In [10]:
all_data['date'] = pd.to_datetime(all_data['date'])

In [11]:
status = pd.read_excel("../Combining_Multiple_Files/data/customer-status.xlsx")
status

,account number,name,status
0,740150,Barton LLC,gold
1,714466,Trantow-Barrows,silver
2,218895,Kulas Inc,bronze
3,307599,"Kassulke, Ondricka and Metz",bronze
4,412290,Jerde-Hilpert,bronze
5,729833,Koepp Ltd,silver
6,146832,Kiehn-Spinka,silver
7,688981,Keeling LLC,silver
8,786968,"Frami, Hills and Schmidt",silver
9,239344,Stokes LLC,gold


#### Merge all_data and status

In [12]:
all_data_status = pd.merge(all_data, status, how='left')
all_data_status.head()

,account number,name,sku,quantity,unit price,ext price,date,status
0,383080,Will LLC,B1-20000,7,33.69,235.83,2014-02-01 09:04:59,NaN
1,412290,Jerde-Hilpert,S1-27722,11,21.12,232.32,2014-02-01 11:51:46,bronze
2,412290,Jerde-Hilpert,B1-86481,3,35.99,107.97,2014-02-01 17:24:32,bronze
3,412290,Jerde-Hilpert,B1-20000,23,78.90,1814.70,2014-02-01 19:56:48,bronze
4,672390,Kuhn-Gusikowski,S1-06532,48,55.82,2679.36,2014-02-02 03:45:20,silver


#### Doing Sanity Checks - investigating specific accounts

In [13]:
all_data_status[all_data_status["account number"]==737550].head()

,account number,name,sku,quantity,unit price,ext price,date,status
15,737550,"Fritsch, Russel and Anderson",S1-47412,40,51.01,2040.40,2014-02-05 01:20:40,NaN
25,737550,"Fritsch, Russel and Anderson",S1-06532,34,18.69,635.46,2014-02-07 09:22:02,NaN
66,737550,"Fritsch, Russel and Anderson",S1-27722,15,70.23,1053.45,2014-02-16 18:24:42,NaN
78,737550,"Fritsch, Russel and Anderson",S2-34077,26,93.35,2427.10,2014-02-20 18:45:43,NaN
80,737550,"Fritsch, Russel and Anderson",S1-93683,31,10.52,326.12,2014-02-21 13:55:45,NaN


##### This account number was not in our status file, hence the NaN's. For this specific case, let's label all missing accounts as bronze - use the fillna

In [14]:
all_data_status['status'].fillna('bronze',inplace=True)
all_data_status.head()

,account number,name,sku,quantity,unit price,ext price,date,status
0,383080,Will LLC,B1-20000,7,33.69,235.83,2014-02-01 09:04:59,bronze
1,412290,Jerde-Hilpert,S1-27722,11,21.12,232.32,2014-02-01 11:51:46,bronze
2,412290,Jerde-Hilpert,B1-86481,3,35.99,107.97,2014-02-01 17:24:32,bronze
3,412290,Jerde-Hilpert,B1-20000,23,78.90,1814.70,2014-02-01 19:56:48,bronze
4,672390,Kuhn-Gusikowski,S1-06532,48,55.82,2679.36,2014-02-02 03:45:20,silver


In [15]:
#testing again to see if change went through
all_data_status[all_data_status["account number"]==737550].head()

,account number,name,sku,quantity,unit price,ext price,date,status
15,737550,"Fritsch, Russel and Anderson",S1-47412,40,51.01,2040.40,2014-02-05 01:20:40,bronze
25,737550,"Fritsch, Russel and Anderson",S1-06532,34,18.69,635.46,2014-02-07 09:22:02,bronze
66,737550,"Fritsch, Russel and Anderson",S1-27722,15,70.23,1053.45,2014-02-16 18:24:42,bronze
78,737550,"Fritsch, Russel and Anderson",S2-34077,26,93.35,2427.10,2014-02-20 18:45:43,bronze
80,737550,"Fritsch, Russel and Anderson",S1-93683,31,10.52,326.12,2014-02-21 13:55:45,bronze


In [16]:
pd.__version__

'0.25.1'

#### Change 'status' to category data type

In [17]:
all_data_status["status"] = all_data_status["status"].astype("category")

In [18]:
all_data_status.head()

,account number,name,sku,quantity,unit price,ext price,date,status
0,383080,Will LLC,B1-20000,7,33.69,235.83,2014-02-01 09:04:59,bronze
1,412290,Jerde-Hilpert,S1-27722,11,21.12,232.32,2014-02-01 11:51:46,bronze
2,412290,Jerde-Hilpert,B1-86481,3,35.99,107.97,2014-02-01 17:24:32,bronze
3,412290,Jerde-Hilpert,B1-20000,23,78.90,1814.70,2014-02-01 19:56:48,bronze
4,672390,Kuhn-Gusikowski,S1-06532,48,55.82,2679.36,2014-02-02 03:45:20,silver


In [19]:
all_data_status.dtypes

account number             int64
name                      object
sku                       object
quantity                   int64
unit price               float64
ext price                float64
date              datetime64[ns]
status                  category
dtype: object

#### Sorting the dataframe by status column

In [20]:
all_data_status.sort_values('status').head()

,account number,name,sku,quantity,unit price,ext price,date,status
0,383080,Will LLC,B1-20000,7,33.69,235.83,2014-02-01 09:04:59,bronze
194,604255,"Halvorson, Crona and Champlin",B1-33087,28,61.35,1717.80,2014-03-21 05:41:09,bronze
197,604255,"Halvorson, Crona and Champlin",S2-00301,35,24.33,851.55,2014-03-21 20:12:32,bronze
201,527099,Sanford and Sons,B1-53102,17,57.52,977.84,2014-03-22 17:29:49,bronze
202,737550,"Fritsch, Russel and Anderson",S2-11481,47,49.22,2313.34,2014-03-22 23:44:18,bronze


#### The above operation sorted in alphabetical order
#### Let's change so that it sorts by our own customer order (the olympic ordering)

In [21]:
all_data_status["status"].cat.set_categories([ "gold","silver","bronze"],inplace=True)

In [22]:
all_data_status.sort_values('status').head()

,account number,name,sku,quantity,unit price,ext price,date,status
265,239344,Stokes LLC,S1-06532,34,71.51,2431.34,2014-01-04 11:34:58,gold
117,141962,Herman LLC,S2-10342,49,31.39,1538.11,2014-03-02 23:08:02,gold
337,141962,Herman LLC,S2-34077,44,90.23,3970.12,2014-01-20 07:27:00,gold
242,239344,Stokes LLC,B1-05914,-1,57.02,-57.02,2014-03-31 06:08:18,gold
103,740150,Barton LLC,S1-30248,21,14.05,295.05,2014-02-28 02:29:03,gold


#### Get some summary info on the status

In [23]:
all_data_status["status"].describe()

count        384
unique         3
top       bronze
freq         172
Name: status, dtype: object

#### Some more useful info about the data

In [24]:

all_data_status.groupby(["status"])["quantity","unit price","ext price"].agg([np.sum,np.mean, np.std])

quantity                       unit price                        \
            sum       mean        std        sum       mean        std   
status                                                                   
gold       1755  24.375000  14.575145    3868.12  53.723889  28.740080   
silver     3198  22.842857  14.512843    8018.18  57.272714  26.556242   
bronze     4406  25.616279  14.136071    9867.84  57.371163  26.857370   

        ext price                            
              sum         mean          std  
status                                       
gold     97340.01  1351.944583  1182.657312  
silver  184804.51  1320.032214  1086.384051  
bronze  253350.14  1472.965930  1116.683843